##### 1. torchvision 概览

> torchvision 是 torch 官方提供的视觉任务工具包，包含用于图像与视频数据的常用数据集加载、预训练模型、图像变换（transform）以及可视化工具。torchvision 独立于 torch，需要专门安装，torchvision 主要包含以下四部分：

- 1）torchvision.models: 模型包括 分类、检测、分割、视频理解四大类：

- - 1.1）在分类方面提供深度学习中各种经典的网络结构，如：Alex-Net、VGG、ResNet、Inception 等

- - 1.2）检测方面提供了常见的检测用的网络结构，FasterRCNN、RetinaNet、MaskRCNN 等

- - 1.3）分割方面提供了常见的分割用的网络结构，FCN, DeepLabV3, LRASPP 等

- - 1.4）视频理解方面不打算涉及，暂时不做学习

- 2）torchvision.datasets：提供常用的数据集，设计上继承 torch.utils.data.Dataset，主要包含分类、检测、分割、视频理解四大类常见数据集：

- - 2.1）分类方面主要包括：MNIST、FashionMNIST、CIFAR10/100、ImageNet、SVHN、Places365 等

- - 2.2）检测、分割方面主要包含：VOC、COCO、Cityscapes、CelebA、Caltech101/256 等

- - 2.3）视频理解方面不打算涉及，暂时不做学习

- 3）torchvision.transforms：提供常用的数据预处理操作，主要包括对 Tensor 及 PIL Image 对象的操作，新版的 transforms 有两套接口：

- - 3.1）传统版 (transforms)：操作 PIL 图像或 Tensor

- - 3.2）新版 (transforms.v2)：统一操作 tv_tensors（带标签的张量类型），支持批处理和自动类型推断，更安全且更快

- 4）torchvision.utils：工具类，如保存张量作为图像到磁盘，给一个小批量创建一个图像网格，该模块使用不太多，等用到会进行更新

In [39]:
# 与 torch 主体完全兼容，但安装时需要与 torch 版本和 CUDA 版本匹配，要查询 torch 和 torchvision 的版本，可以使用下面语句
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.8.0+cu126
0.23.0+cu126


##### 2. torchvision.models 的学习

> 1）torchvision.models: 模型包括 分类、检测、分割、视频理解四大类：

- - 1.1）在分类方面提供深度学习中各种经典的网络结构，如：Alex-Net、VGG、ResNet、Inception 等

- - 1.2）检测方面提供了常见的检测用的网络结构，FasterRCNN、RetinaNet、MaskRCNN 等

- - 1.3）分割方面提供了常见的分割用的网络结构，FCN, DeepLabV3, LRASPP 等

- - 1.4）视频理解方面不打算涉及，暂时不做学习

> torchvision.models 里面包含了很多的常见的模型，可以直接的加载使用，也可以直接加载预训练模型在自己想进行的任务上微调使用

- 1）通过 pretrined=True 可以加载预训练模型，pretrained 默认值是 False，不赋值和赋值 False 效果一样，选择了 pretrained 为 True 的话，点击运行会执行下载预训练参数的操作，不加载预训练模型没有这步操作，可以直接只使用网络架构

- 2）预训练模型使用上面的参数进行下载的时候现在会出现警告，主要是因为 torch 进行了更新，现在调用预训练模型的方法是在参数 weights 上指定调用的预训练模型的名字，参考下面的写法

- 3）直接输出当前的模型或者遍历输出其参数矩阵的名字和大小能检查当前的网络的形状，可以使用 torchsummary 库函数下的 summary 包来检查网络的结构，更加的直观

In [40]:
import torchvision.models as models
from torchsummary import summary as sumy

# torch.device 是一个类，表示一个设备或者对象
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# resnet18 = models.resnet18(pretrained=True) # 旧版本写法会出现警告，该方法将被淘汰，不建议使用
resnet18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
alexnet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
squeezenet = models.squeezenet1_0(weights=models.SqueezeNet1_0_Weights.IMAGENET1K_V1)
densenet = models.densenet161(weights=models.DenseNet161_Weights.IMAGENET1K_V1)
resnet152 = models.resnet152(weights=None)

# 可以使用该方法打印加载的预训练模型的参数
# 第一次接触 model.named_parameters()，他是 torch.nn.Module 类的一个方法。
# 它返回一个可迭代对象（generator），其中每个元素都是一个二元组 (name, parameter)，下面输出前十层的名字和参数矩阵的 shape 展示
print("Model architecture (show first 10 layers):")
for i, (name, param) in enumerate(resnet18.named_parameters()):
    print(f"Layer {i}: ", name, param.shape)
    if i == 9: break

# .to() 是 torch.Tensor 类的一个实例方法
# 当整个模型 .to() 的时候，此时 .to() 是 torch.nn.Module 类 的一个方法，它会自动遍历所有子层的参数和缓冲区（buffers），把它们都放到目标设备上
vgg16 = vgg16.to(device)

# summary 函数的参数：model：要查看的模型对象、input_size：输入的张量尺寸、device：指定设备上生成 summary，其他参数不重要用到可查
# 输出的 Output Shape 的第一个维度为 -1 代表是任意 batch_size 大小
sumy(vgg16, input_size=(3, 224, 224), device=str(device))


Model architecture (show first 10 layers):
Layer 0:  conv1.weight torch.Size([64, 3, 7, 7])
Layer 1:  bn1.weight torch.Size([64])
Layer 2:  bn1.bias torch.Size([64])
Layer 3:  layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
Layer 4:  layer1.0.bn1.weight torch.Size([64])
Layer 5:  layer1.0.bn1.bias torch.Size([64])
Layer 6:  layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
Layer 7:  layer1.0.bn2.weight torch.Size([64])
Layer 8:  layer1.0.bn2.bias torch.Size([64])
Layer 9:  layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6    

> 下面介绍常见的模型加载和保存的方法

- 1）vgg16.state_dict() 得到模型的参数

- 2）加载本地的模型参数使用方法 vgg16.load_state_dict()

- 3）需要结合加载文件转化为字典的函数 torch.load_dict() 一起使用，也就是 vgg16.load_state_dict(torch.load_dict(xxxx.pth))

- 4）保存模型的参数需要得到模型和的字典，然后使用 torch.save() 函数来保存，torch.save(vgg16.state_dict(), "xxxx.pth")

- 5）torch.save() 可以保存训练过程中的其他参数，方便中断之后继续训练，保存的时候写成键值对的形式即可，这样的保存形式加载的时候需要指定对应的键

In [41]:
vgg16_state_dict = vgg16.state_dict()
resnet18_state_dict = resnet18.state_dict()

torch.save(vgg16_state_dict, "./checkpoints/vgg16_state_dict.pth")
torch.save(resnet18_state_dict, "./checkpoints/resnet18_state_dict.pth")

vgg16.load_state_dict(torch.load("./checkpoints/vgg16_state_dict.pth"))

torch.save({
    "epoch": None,
    "model_state_dict": vgg16_state_dict,
    "optimizer_state_dict": None, # 假设训练使用的是带动量的 SGD 优化器，优化器参数保存了之前的动量，所以要从断点继续训练，这个参数是必须保存的
    "lr": None, # 断点恢复训练必须保存
    "loss": None
}, "./checkpoints/vgg16.pth")

vgg16.load_state_dict(torch.load("./checkpoints/vgg16.pth")["model_state_dict"])

<All keys matched successfully>

- 1）但是正常的使用过程中可能会修改模型的层数，想要接着使用预训练的参数就会出现无法加载的问题，所以必须在 pth 文件里面删掉或者加上相应的修改的层

- 2）加入现在 resnet18 的分类任务变为了分类为 10 类，就需要对最后的全连接层进行修改，层的名字和 shape 都是匹配的话，成功的加载，名字不匹配但是形状匹配的话，strict=False 会忽略错误，名字匹配但是形状不匹配，就会报错

- 3）pretrained_state_dict = {k : v for k, v in pretrained_state_dict.items() if k in model_state_dict} 这个写法很显然只会比较键是不是完全一样，即使修改了形状也不会关心，所以需要加上判断，v.shape 和 model_state_dict[k].shape 要是一样的才行，这样的话，只会从预训练模型里面加载确定没有修改的层

In [42]:
import torch.nn as nn

model = models.resnet18(weights=None) # 加载空模型

model.fc = nn.Linear(512, 10) # 修改全连接层的大小

pretrained_state_dict = resnet18.state_dict() # 预训练模型参数
model_state_dict = model.state_dict() # 加载的模型参数

# 验证键是一样的，也就是层没有改变
print(pretrained_state_dict.keys() == model_state_dict.keys())
print(pretrained_state_dict["fc.bias"].size())

pretrained_state_dict = {k : v for k, v in pretrained_state_dict.items() if k in model_state_dict and v.shape == model_state_dict[k].shape}

# res = model.load_state_dict(torch.load('./resnet18_state_dict.pth'), strict=False) 报错，因为无法对应，原本的预训练模型的全连接层的大小为 [512, 1000]，现在被我修改为了 [512, 10]
res = model.load_state_dict(pretrained_state_dict, strict=False) # 完美的加载，验证了自己的想法，现在删掉了全连接层的参数，其他的参数对应上了

print("Missing keys:", res.missing_keys) # 模型中存在，但是在预训练参数里面找不到
print("Unexpected keys:", res.unexpected_keys) # 预训练参数中存在模型中没定义，这个不会遇到，因为我强制的比对了键值对，模型中没定义的一定会被删掉的

print(model.fc.weight.mean(), model.fc.weight.std())  # 随机分布
print(resnet18.fc.weight.mean(), resnet18.fc.weight.std())  # 不同分布


True
torch.Size([1000])
Missing keys: ['fc.weight', 'fc.bias']
Unexpected keys: []
tensor(-0.0004, grad_fn=<MeanBackward0>) tensor(0.0253, grad_fn=<StdBackward0>)
tensor(5.8502e-08, grad_fn=<MeanBackward0>) tensor(0.0695, grad_fn=<StdBackward0>)


> 上面简单的介绍了模型的加载和报错，和加载预训练参数的方法等，这个是 torchvision.models 库函数里面常见的操作了，总结下列函数：

- 1）models.model() 加载名字为 model 的模型，一般的参数为 weights，代表加载预训练参数，不想加载的话初始化为 weights=None

- 2）model.named_parameters() 返回一个生成器，每个元素是 (name, parameter) 二元组

- 3）summary() 来自 torchsummary 或 torchinfo 包，需单独安装，能模拟一张输入图片走完整个网络，输出表格信息（每层输出尺寸、参数量、总参数数等）

- 4）torch.save() 保存模型，可以保存任意 Python 对象包括模型参数、优化器状态、epoch 等

- 5）torch.load_state_dict() 加载模型

- 6）torch.load() 加载文件转化为字典

- 7）model.state_dict() 返回全部参数加上缓冲区（不带梯度信息），named_parameters() 返回仅可训练参数（带梯度）

##### 3. torchvision.datasets、torchvision.transforms 的学习

> - 2）torchvision.datasets：提供常用的数据集，设计上继承 torch.utils.data.Dataset，主要包含分类、检测、分割、视频理解四大类常见数据集：

- - 2.1）分类方面主要包括：MNIST、FashionMNIST、CIFAR10/100、ImageNet、SVHN、Places365 等

- - 2.2）检测、分割方面主要包含：VOC、COCO、Cityscapes、CelebA、Caltech101/256 等

- - 2.3）视频理解方面不打算涉及，暂时不做学习

> - 3）torchvision.transforms：提供常用的数据预处理操作，主要包括对 Tensor 及 PIL Image 对象的操作，新版的 transforms 有两套接口：

- - 3.1）传统版 (transforms)：操作 PIL 图像或 Tensor

- - 3.2）新版 (transforms.v2)：统一操作 tv_tensors（带标签的张量类型），支持批处理和自动类型推断，更安全且更快

In [43]:
from torchvision import datasets
from torchvision import transforms as T

transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

# root 数据集的存放路径，没有的话，会自动存放到该文件夹
# train 是否加载训练集，否则的话就是测试集
# transform torchvision.transforms 包含对图像进行变换的函数，ToTensor()、Normalize()
# target_transform torchvision.transforms 对标签变换的函数，一般不使用
# download 如果本地没有数据集，是否从网上下载
mnist = datasets.MNIST("data/", download=True, train=False, transform=transform)

img = mnist[0][0]
label = mnist[0][1]
to_img = T.ToPILImage()
display(to_img(img))
print(label)

7


- 1）当第一次调用 datasets.MNIST(download=True) 时，它会自动执行以下步骤检查 raw 文件夹如果没有 .gz 文件，就从官网（yann.lecun.com）下载，自动解压 .gz 文件

- 2）存入 data/MNIST/processed/，解析二进制数据为 Tensor，把像素值解析成 torch.Tensor 格式，把标签解析成 torch.LongTensor，缓存结果，将解析好的数据保存为两个 .pt 文件

- 3）training.pt 包含 (train_images, train_labels)，参考上面是输出 dataset 是一个存储文件，保存了 Tensor 类型的文件和对应的标签，test.pt 包含 (test_images, test_labels)

- 4）因为 datasets.MNIST 继承自 torch.utils.data.Dataset，实现了 \_\_getitem\_\_ 和 \_\_len\_\_ 两个函数，可以直接 train_loader = DataLoader(train_data, batch_size=64, shuffle=True)，不需要也不能进行其他任何多余的操作，这个是 torchvision.datasets 的很大的局限性，只能加载指定名字的指定格式的数据集，想要将自己的数据集转化为 DataLoader 还是需要使用 torch.utils.data.Dataset 实现 \_\_getitem\_\_ 和 \_\_len\_\_ 函数，然后转化为 Dataset 类型的数据集，再进行 DataLoader 的批量加载

In [44]:
from torch.utils.data import DataLoader

train_loader = DataLoader(mnist, batch_size=64, shuffle=True)

train_iter = iter(train_loader)
batch_img, batch_label = next(train_iter)

print(batch_img.shape, batch_label.shape)
for batch_idx, (imgs, labels) in enumerate(train_loader):
    print(batch_idx, imgs.shape, labels.shape)
    if batch_idx == 5: break

# train_data.data 输出所有图片，train_data.targets 输出所有的标签
print(mnist.data.shape, mnist.targets.shape)

image, label = mnist[0]
print(image.shape, label)

torch.Size([64, 1, 28, 28]) torch.Size([64])
0 torch.Size([64, 1, 28, 28]) torch.Size([64])
1 torch.Size([64, 1, 28, 28]) torch.Size([64])
2 torch.Size([64, 1, 28, 28]) torch.Size([64])
3 torch.Size([64, 1, 28, 28]) torch.Size([64])
4 torch.Size([64, 1, 28, 28]) torch.Size([64])
5 torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([10000, 28, 28]) torch.Size([10000])
torch.Size([1, 28, 28]) 7


In [45]:
from torchvision import transforms as T

# 包含了常见的图像处理的函数
dir(T)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

> 常见的图像处理函数的总结，用到可以自行查找

- 1）Compose()：用来管理所有的 transforms 操作

- 2）ToTensor()：把图片数据转换成张量并转化范围在 [0,1] 区间内

- 3）Normalize(mean, std)：归一化

- 4）Resize(size)：输入的 PIL 图像调整为指定的大小，参数可以为 int 或 int 元组

- 5）CenterCrop(size)：将给定的 PIL Image 进行中心切割，得到指定 size 的 tuple

- 6）RandomCrop(size, padding=0)：随机中心点切割

- 7）RandomHorizontalFlip(size, interpolation=2)：将给定的 PIL Image 随机切割，再 resize

- 8）RandomHorizontalFlip()：随机水平翻转给定的 PIL Image

- 9）RandomVerticalFlip()：随机垂直翻转给定的 PIL Image

- 10）ToPILImage()：将 Tensor 或 numpy.ndarray 转换为 PIL Image

- 11）FiveCrop(size)：将给定的 PIL 图像裁剪成 4 个角落区域和中心区域

- 12）Pad(padding, fill=0, padding_mode=‘constant’)：对 PIL 边缘进行填充

- 13）RandomAffine(degrees, translate=None, scale=None)：保持中心不变的图片进行随机仿射变化

- 15）RandomApply(transforms, p=0.5)：随机选取变换